In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install scikit-learn
!pip3 install joblib

In [ ]:
import pandas as pd
import numpy as np

# Apartado 5

En resumen, el método elegido es el de ``SVM`` y la técnica de escalado es ``StandardScaler``. Estos serán por lo tanto los métodos que utilicemos para realizar la evalución del rendimiento futuro, como se muestra en el siguiente código:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import TimeSeriesSplit

# Carga los datos de entrenamiento en un DataFrame
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")

# Selecciona solo las columnas numéricas
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns

# Nos aseguramos de que 'energy' no está en las variables numéricas, ya que es la variable objetivo
numerical_vars = numerical_vars.drop('energy')

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(wind_ava[numerical_vars], wind_ava['energy'], test_size=1/3, random_state=100472166, shuffle=False)

# Crea el escalador
scaler = StandardScaler()

# Ajusta el escalador a las características del conjunto de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Transforma las características del conjunto de prueba usando el mismo escalador
X_test_scaled = scaler.transform(X_test)

# Define el modelo SVM con los hiperparámetros seleccionados
best_model = SVR(C=1000.0, coef0= 0, gamma= 'scale', kernel= 'rbf', shrinking= True)

# Define el número de splits
tscv = TimeSeriesSplit(n_splits=3)

# Itera a través de los splits de TimeSeriesSplit
for train_index, test_index in tscv.split(X_train_scaled):
    X_train_inner, X_val = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_inner, y_val = y_train.iloc[train_index], y_train.iloc[test_index]

    # Ajusta el modelo SVM en el conjunto de entrenamiento más pequeño
    best_model.fit(X_train_inner, y_train_inner)

    # Hace predicciones en el conjunto de validación
    y_val_pred = best_model.predict(X_val)

    # Pone los resultados negativos a 0, ya que no tendrían sentido
    y_val_pred = np.maximum(y_val_pred, 0)

    # Calcula los errores en el conjunto de validación
    mse = mean_squared_error(y_val, y_val_pred)
    rmse = np.sqrt(mse)
    print(f"MSE en el conjunto de validación: {mse}")
    print(f"RMSE en el conjunto de validación: {rmse}")
    print("------------------------")

# Hace predicciones en el conjunto de prueba
y_test_pred = best_model.predict(X_test_scaled)
y_test_pred = np.maximum(y_test_pred, 0)

# Calcula los errores en el conjunto de prueba
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
print(f"MSE en el conjunto de prueba: {mse_test}")
print(f"RMSE en el conjunto de prueba: {rmse_test}")

El ``MSE`` obtenido es de 159004. Como era de esperar, al entrenarlo con todos los datos disponibles el error es algo menor que el resultado del ajuste de hiperparámetros (159220). El `` RMSE `` en cambio es de 398. Todos estos valores son por lo tanto los más bajos que hemos obtenido hasta ahora

In [ ]:
from joblib import dump

# Guarda el modelo en un archivo
with open('modelo_final.pkl', 'wb') as file:
    dump(best_model, file)

Ahora utilizaremos este modelo para realizar predicciones con los datos de la competición y las guardaremos en el fichero `` predicciones.csv ``:

In [ ]:
from joblib import load

# Carga los datos de competición en un DataFrame
competition_data = pd.read_csv('wind_comp.csv.gz', compression="gzip")

# Almacenamos las variables numéricas
numerical_vars_competition = competition_data.select_dtypes(include=['int64', 'float64']).columns

# Escalado de los datos de la competición
X_competition_scaled = scaler.transform(competition_data[numerical_vars_competition])

# Cargar el modelo final
with open('modelo_final.pkl', 'rb') as file:
    loaded_model = load(file)

# Hace predicciones en el conjunto de datos de la competición
competition_predictions = loaded_model.predict(X_competition_scaled)
competition_predictions = np.maximum(competition_predictions, 0)

# Guarda las predicciones en un archivo CSV
pd.DataFrame(competition_predictions).to_csv('predicciones.csv', index=False, header=False)